# AWS S3 활용하기

tensorflow에서 제공해주는 데이터가 아닌 직접 수집한 데이터를 활용하는 경우가 더 많습니다.<br>
본 예제에서는 S3에 저장된 kaggle에서 제공하는 개와 고양이 사진 데이터를 활용하여 모델 학습을 진행합니다.

In [ ]:
from kfp import components, dsl
from kfp.components import InputPath, OutputPath

## S3 정보

예제 진행에 앞서 S3를 사용하기 위해 필요한 정보들을 가져옵니다.
access key 및 secret access key 는 다음을 참고합니다.<br>
참고 : https://docs.aws.amazon.com/ko_kr/IAM/latest/UserGuide/id_credentials_access-keys.html

In [ ]:
    aws_access_key = 'xxxxxxxxxxxxxxxxx'
    aws_secret_access_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
    bucket_name = [bucket name]
    bucket_dir_name = [bucket directory] #data 저장 경로

access key, secret access key, bucket 이름, bucket 내 경로 정보를 가져옵니다.

## S3에서 데이터 가져오기

In [ ]:
def train_data_load_from_s3(
        output_dataset_train_data: OutputPath(str)
):
    import boto3
    import os
    aws_access_key = 'xxxxxxxxxxxxxxxxx'
    aws_secret_access_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
    bucket_name = [bucket name]
    bucket_dir_name = [bucket directory] #data 저장 경로

    s3r = boto3.resource('s3', aws_access_key_id=aws_access_key,
                         aws_secret_access_key=aws_secret_access_key)
    bucket = s3r.Bucket(bucket_name)
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key,
                      aws_secret_access_key=aws_secret_access_key)

    count_dog = 0
    count_cat = 0

    os.makedirs(output_dataset_train_data)
    for object in bucket.objects.filter(Prefix=bucket_dir_name):
        if 'cat' in object.key :
            temp = f'{output_dataset_train_data}/cat_{count_cat}.jpg'
            s3.download_file(bucket_name, object.key, temp)
            count_cat += 1
        elif 'dog' in object.key :
            temp = f'{output_dataset_train_data}/dog_{count_dog}.jpg'
            s3.download_file(bucket_name, object.key, temp)
            count_dog += 1


train_data_load_op = components.create_component_from_func(
    train_data_load_from_s3, base_image='python:3.9',
    packages_to_install=['boto3']
)

S3 정보를 바탕으로 boto3 패키지를 활용해 데이터를 가져옵니다. 컴포넌트 간 주고 받는 데이터 경로에 S3에서 가져온 데이터를 저장합니다.

## 데이터 확인

In [ ]:
def checked_data_image_list(
        pre_data:InputPath(str)
):
    import os
    file_list = os.listdir(pre_data)
    dog_list = []
    cat_list = []
    for i, item in enumerate(file_list):
        if 'cat' in item :
            dog_list.append(item)
        elif 'dog' in item :
            cat_list.append(item)

    print(f'train_dog_image_num : {len(dog_list)}')
    print(f'train_cat_image_num : {len(cat_list)}')

checked_data_image_list_op = components.create_component_from_func(
    checked_data_image_list, base_image='python:3.9'
)

데이터가 정상적으로 저장되었는지 확인합니다.

## 데이터셋 생성

In [ ]:
def data_generation(
        pre_data: InputPath(str),
        train_data: OutputPath('Dataset')
):
    from skimage import io
    from skimage.transform import resize
    import numpy as np
    import os
    import pandas as pd
    import pickle

    file_list = os.listdir(pre_data)
    train_list = []
    label_list = []
    for i, item in enumerate(file_list):
        temp = f'{pre_data}/{item}'
        img = io.imread(temp)
        resize_img = resize(img,(224,224))
        resize_img = np.array(resize_img)
        train_list.append(resize_img)
        if 'cat' in item :
            label_list.append([1,0])
        elif 'dog' in item :
            label_list.append([0,1])


    df = pd.DataFrame(columns=['image', 'label'])
    for i, image in enumerate(train_list):
        df.loc[i] = ({'image': image, 'label': label_list[i]})

    with open(train_data, 'wb') as f:
        pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

data_generation_op = components.create_component_from_func(
    data_generation, base_image='python:3.9',
    packages_to_install=['numpy', 'scikit-image', 'pandas']
)

저장된 이미지 데이터를 읽어와서 전처리를 통해 학습할 수 있는 데이터셋을 생성합니다. <br>
생성된 데이터셋은 pickle 파일로 저장합니다.

## 모델 생성

In [ ]:
def model_generation(
        pretrain_model : OutputPath('TFModel')
) :
    from keras.applications import ResNet50
    from keras.models import Sequential

    model = Sequential()
    model.add(ResNet50(include_top=True, weights=None, input_shape=(224, 224, 3), classes=2))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.save(pretrain_model)

model_generation_op = components.create_component_from_func(
    model_generation, base_image='tensorflow/tensorflow'
)

이미지 분류 모델로 ResNet50을 활용합니다.

## 학습하기

In [ ]:
def train_model(
        train_dataset: InputPath('Dataset'),
        pre_model: InputPath('TFModel'),
        trained_model : OutputPath('TFModel')
) :
    import pickle
    from tensorflow import keras
    import numpy as np
    import pandas as pd

    with open(train_dataset, 'rb') as file:
        tr_data = pickle.load(file)

    images = []
    labels = []
    for i, item in enumerate(tr_data['image']) :
        images.append(item)
        labels.append(tr_data['label'][i])

    images = np.array(images)
    labels = np.array(labels)
    model = keras.models.load_model(pre_model)

    model.fit(images, labels, epochs=200)
    model.save(trained_model)

train_result_op = components.create_component_from_func(
    train_model,
    base_image='tensorflow/tensorflow',
    packages_to_install=['pandas==1.4.2']
)

pickle 형태로 저장된 데이터셋과 ResNet50 모델을 입력받아 학습을 진행합니다.

## 파이프라인 생성

In [ ]:
@dsl.pipeline(name='example data load from s3 and train')
def aws_dog_cat_classification_pipeline():
    train_data_load_task = train_data_load_op()
    checked_data_image_list_task = checked_data_image_list_op(train_data_load_task.outputs['output_dataset_train_data'])
    model_generation_task = model_generation_op()
    data_generation_task = data_generation_op(train_data_load_task.outputs['output_dataset_train_data'])
    train_task = train_result_op(
        data_generation_task.outputs['train_data'],
        model_generation_task.outputs['pretrain_model']
    )

In [ ]:
kfp.compiler.Compiler().compile(aws_dog_cat_classification_pipeline, 'data_load_from_s3.yaml')